In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [ ]:
webdriver_path = 'geckodriver'
binary_path = "C:\Program Files (x86)\Mozilla Firefox\firefox.exe"
firefox_options = Options()
firefox_options.add_argument('--headless')
firefox_options.add_argument('--no-sandbox')
driver = webdriver.Firefox(service=Service(webdriver_path), options=firefox_options)

In [ ]:
url = "https://qalampir.uz"

In [ ]:
driver.get(url+"/uz")
html_content = driver.page_source
soup = BeautifulSoup(html_content, "html.parser")
items = soup.find_all("a",class_="nav-link")
menu = []

for item in items:
    menu.append(url+"/uz"+item["href"])
    print(url+"/uz"+item["href"])

In [ ]:
category_name = [category.split("/")[-1] for category in menu]
category_name

In [ ]:
with open("qalampir_uz.txt", "+a") as file:
    
    separator = "~~~#~~~"
    
    for item, category in zip(menu, category_name):
        print(f"{item} scrap boshlandi...")
        driver.get(item)
        page = 0
        
        while True:
            page+=1
            
            try:
                element = driver.find_element(By.XPATH,"//button[@class=\"refresh-btn\"]")
                element = element.click()
                
                if page%100==0:
                    print(f"{page} ta sahifa scrap qilindi")
                
            except NoSuchElementException:
                break
        
        print(f"{item} scrap tugadi\n\n")
        
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        items = soup.find_all("a",class_="news-card")
        count = 0
        
        for item in items:
            href = url+item["href"]
            file.write(f"{href}{separator}{category}\n")

In [ ]:
link = "https://qalampir.uz/uz/news/mirziyeevning-administratsiyasi-putin-k-anday-khabarni-yetkazganini-ochik-ladi-84947"
driver.get(link)
html_content = driver.page_source
soup = BeautifulSoup(html_content, "html.parser")

In [ ]:
with open("qalampir.txt","+a") as wfile:
    with open("qalampir_uz.txt") as rfile:
        
        for items in rfile:
            link, category = items.split(separator)
            print(link)
            driver.get(link)
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, "html.parser")

            title = soup.find("h1",class_="text").text
            content = "".join([p.text for p in soup.find("div", class_ = "row g-4 my-main-content").find_all("p")])

            wfile.write(f"{title}{separator}{content}{separator}{category}\n")

In [ ]:
def clean_element(element):
    return element.replace("\n","").strip()


dataset = pd.DataFrame(columns=['title', 'content', 'category'])

with open("qalampir.txt") as file:
    for news in file:
        
        news = news.split(separator)
        dataset = dataset.append([{'title':clean_element(news[0]), 'content':clean_element(news[1]), 'category':clean_element(news[2])}],ignore_index = True)
        
dataset.head()

In [ ]:
dataset.to_csv("qalampir.csv", encoding='utf-8', index = False)